In [1]:
!cd /Users/lc/Developer/kuzu &&  make clean-python-api && make release NUM_THREADS=20 

rm -rf tools/python_api/build
cd external && \
	mkdir -p build && \
	cd build && \
	cmake    -DCMAKE_BUILD_TYPE=Release ../ && \
	cmake --build . --config Release -- -j 20
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/lc/Developer/kuzu/external/build
[100%] Built target apache_arrow
mkdir -p build/release && \
	cd build/release && \
	cmake    -DCMAKE_BUILD_TYPE=Release ../.. && \
	cmake --build . --config Release -- -j 20
-- pybind11 v2.11.0 dev1
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/lc/Developer/kuzu/build/release
[  1%] Built target utf8proc
[  1%] Built target kuzu_common_csv_reader
[  2%] Built target kuzu_transaction
[  2%] Built target kuzu_common_arrow
[  2%] Built target kuzu_common_task_system
[  3%] Built target kuzu_common_data_chunk
[  3%] Built target kuzu_catalog
[  4%] Built target kuzu_binder_expression
[  5%] Built target kuzu_common_vector
[  6%] Built target kuzu_binder
[  7%] Built tar

In [1]:
import sys
sys.path.append('/Users/lc/Developer/kuzu/tools/python_api/build')
import kuzu

In [2]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG


dataset = OGB_MAG(root='/Users/lc/Developer/ogb_pytorch', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

In [3]:
import torch

In [4]:
DB_PATH = '/Users/lc/Developer/kuzu_ogb_mag'
db = kuzu.Database(DB_PATH)

In [5]:
conn = kuzu.Connection(db)

In [6]:
fs, gs = db.get_torch_geometric_remote_backend()

In [7]:
from torch_geometric.loader import NeighborLoader
loader_kuzu = NeighborLoader(
    data=(fs, gs),
    num_neighbors={key.edge_type : [8] * 2 for key in gs.get_all_edge_attrs()},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=128,
    input_nodes=('paper', torch.arange(0, data['paper'].num_nodes)),
    input_time=None
)

In [8]:
next(iter(loader_kuzu))

HeteroData(
  paper={
    num_nodes=726,
    id=[726],
    x=[726, 128],
    year=[726],
    y=[726],
    input_id=[128],
    batch_size=128
  },
  institution={
    num_nodes=3,
    id=[3],
    x=[3, 128]
  },
  author={
    num_nodes=2413,
    id=[2413],
    x=[2413, 128]
  },
  field_of_study={
    num_nodes=30,
    id=[30],
    x=[30, 128]
  },
  (paper, has_topic, field_of_study)={ edge_index=[2, 240] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 7] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 3354] },
  (paper, cites, paper)={ edge_index=[2, 2550] },
  (author, writes, paper)={ edge_index=[2, 2456] },
  (paper, rev_writes, author)={ edge_index=[2, 42] }
)

In [1]:
loader_pyg = NeighborLoader(
    data=(fs, gs),
    num_neighbors={key.edge_type : [8] * 2 for key in gs.get_all_edge_attrs()},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=128,
    input_nodes=('paper', torch.arange(0, data['paper'].num_nodes)),
    input_time=None
)

NameError: name 'NeighborLoader' is not defined

In [ ]:
next(iter(loader_kuzu))

In [12]:
gs.csr()

({('paper',
   'has_topic',
   'field_of_study'): tensor([      0,      11,      22,  ..., 7505065, 7505068, 7505078]),
  ('author',
   'affiliated_with',
   'institution'): tensor([      0,       1,       2,  ..., 1043996, 1043997, 1043998]),
  ('institution',
   'rev_affiliated_with',
   'author'): tensor([      0,  283527,  283527,  ..., 1043998, 1043998, 1043998]),
  ('field_of_study',
   'rev_has_topic',
   'paper'): tensor([      0,  519286,  519286,  ..., 7505078, 7505078, 7505078]),
  ('paper',
   'cites',
   'paper'): tensor([       0,    21425,    21425,  ..., 10792672, 10792672, 10792672]),
  ('author',
   'writes',
   'paper'): tensor([      0,       4,       7,  ..., 7145654, 7145658, 7145660]),
  ('paper',
   'rev_writes',
   'author'): tensor([      0,   15533,   15533,  ..., 7145660, 7145660, 7145660])},
 {('paper',
   'has_topic',
   'field_of_study'): tensor([    0,  2048,  2048,  ..., 20480, 20480, 30720]),
  ('author',
   'affiliated_with',
   'institution'): tensor

In [8]:
gs.get_all_edge_attrs()

[EdgeAttr(edge_type=('paper', 'has_topic', 'field_of_study'), layout=<EdgeLayout.COO: 'coo'>, is_sorted=False, size=(736389, 59965)),
 EdgeAttr(edge_type=('author', 'affiliated_with', 'institution'), layout=<EdgeLayout.COO: 'coo'>, is_sorted=False, size=(1134649, 8740)),
 EdgeAttr(edge_type=('institution', 'rev_affiliated_with', 'author'), layout=<EdgeLayout.COO: 'coo'>, is_sorted=False, size=(8740, 1134649)),
 EdgeAttr(edge_type=('field_of_study', 'rev_has_topic', 'paper'), layout=<EdgeLayout.COO: 'coo'>, is_sorted=False, size=(59965, 736389)),
 EdgeAttr(edge_type=('paper', 'cites', 'paper'), layout=<EdgeLayout.COO: 'coo'>, is_sorted=False, size=(736389, 736389)),
 EdgeAttr(edge_type=('author', 'writes', 'paper'), layout=<EdgeLayout.COO: 'coo'>, is_sorted=False, size=(1134649, 736389)),
 EdgeAttr(edge_type=('paper', 'rev_writes', 'author'), layout=<EdgeLayout.COO: 'coo'>, is_sorted=False, size=(736389, 1134649))]

In [4]:
conn = kuzu.Connection(db, num_threads=20)
conn._get_rel_table_names()

[{'name': 'has_topic', 'src': 'paper', 'dst': 'paper'},
 {'name': 'affiliated_with', 'src': 'author', 'dst': 'author'},
 {'name': 'rev_affiliated_with', 'src': 'institution', 'dst': 'institution'},
 {'name': 'rev_has_topic', 'src': 'field_of_study', 'dst': 'field_of_study'},
 {'name': 'cites', 'src': 'paper', 'dst': 'paper'},
 {'name': 'writes', 'src': 'author', 'dst': 'author'},
 {'name': 'rev_writes', 'src': 'paper', 'dst': 'paper'}]

In [5]:
conn._get_rel_property_names('has_topic')

'has_topic src node: paper\nhas_topic dst node: paper\nhas_topic properties: \n'

In [8]:
res =conn.execute("MATCH (a:paper)-[has_topic]-> (b:paper) RETURN offset(id(a)), offset(id(b))")

In [10]:
df = res.get_as_df()

In [13]:
df

,offset(id(a)),offset(id(b))
0,6144,81250
1,6144,87091
2,6144,178455
3,6144,183986
4,6144,208930
...,...,...
10792667,692224,92598
10792668,692224,413209
10792669,692224,471244
10792670,692224,531688


In [26]:
data

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 10792672] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] },
  (paper, has_topic, paper)={}
)

In [24]:
src = torch.LongTensor(df.iloc[:,0])
dst = torch.LongTensor(df.iloc[:,1])
edge_list = torch.stack([src, dst], dim=0)

In [25]:
edge_list

tensor([[  6144,   6144,   6144,  ..., 692224, 692224, 692224],
        [ 81250,  87091, 178455,  ..., 471244, 531688, 686206]])

In [5]:
data

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 10792672] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] }
)

In [6]:
fs = db.get_torch_geometric_remote_backend()

In [24]:
import random
import time

pyg_times = []
kuzu_times = []

n = 100
for i in range(n):
    print(i + 1, "/", n)
    ids = random.sample(range(0, len(data['paper'].x) - 1000), 1)
    start_id = ids[0]
    
    start = time.time()
    d = fs['paper'].x[start_id:start_id + 1000]
    end = time.time()
    kuzu_times.append(end - start)

    start = time.time()
    d = data['paper'].x[start_id:start_id + 1000]
    end = time.time()
    pyg_times.append(end - start)

    

1 / 100
Query time: 0.002570
Fetch time: 0.013129
Convert time: 0.004950
2 / 100
Query time: 0.002453
Fetch time: 0.011969
Convert time: 0.005066
3 / 100
Query time: 0.003132
Fetch time: 0.012392
Convert time: 0.004983
4 / 100
Query time: 0.003172
Fetch time: 0.013273
Convert time: 0.004949
5 / 100
Query time: 0.002481
Fetch time: 0.012160
Convert time: 0.004861
6 / 100
Query time: 0.002436
Fetch time: 0.012085
Convert time: 0.004811
7 / 100
Query time: 0.002363
Fetch time: 0.012742
Convert time: 0.004952
8 / 100
Query time: 0.002479
Fetch time: 0.012333
Convert time: 0.004793
9 / 100
Query time: 0.002459
Fetch time: 0.013155
Convert time: 0.004812
10 / 100
Query time: 0.003084
Fetch time: 0.013645
Convert time: 0.005035
11 / 100
Query time: 0.002466
Fetch time: 0.013179
Convert time: 0.004818
12 / 100
Query time: 0.002465
Fetch time: 0.012473
Convert time: 0.004837
13 / 100
Query time: 0.002457
Fetch time: 0.012009
Convert time: 0.004883
14 / 100
Query time: 0.001826
Fetch time: 0.013

In [25]:
pyg_time = sum(pyg_times) / len(pyg_times)
kuzu_time = sum(kuzu_times) / len(kuzu_times)

print("PyG time: ", pyg_time)
print("Kuzu time: ", kuzu_time)

print(kuzu_time / pyg_time)


PyG time:  2.7034282684326172e-05
Kuzu time:  0.020634567737579344
763.2740982449951
